Resources:

- [cifar10 (Colab)](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/cifar10_tutorial.ipynb#scrollTo=ubQdOyR6FjUB)

- [Iterative Pruning (Colab)](https://pytorch.org/tutorials/intermediate/pruning_tutorial.html)

- [Pruning Experiments (Github)](https://github.com/olegpolivin/pruningExperiments/tree/main)
- [Pruning Experiments (Medium)](https://olegpolivin.medium.com/experiments-in-neural-network-pruning-in-pytorch-c18d5b771d6d)
- [Sparsifying Regularizer (Github)](https://github.com/dizam92/pyTorchReg)

Components
- data loading + preprocessing (/utils/data_utils.py ??)

In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from regularizers.regularizer import *

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time

from utils.data_utils import preprocess
from utils.training_utils import training_loop
from utils.training_utils import package_model_components
from utils.training_utils import calculate_regularizer_metrics
from models.model import Net

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [10]:
testloader,trainloader = preprocess()

Files already downloaded and verified
Files already downloaded and verified


In [11]:
loss_across_epochs = []
latency_across_epochs = []
list_of_regularizers = [NullRegularizer,L1Regularizer]

In [13]:
from regularizers.regularizer import NullRegularizer

NullRegularizer(model=Net, lambda_reg=10**-2)

# model_components = package_model_components(Net,list_of_regularizers)

NameError: name 'reload' is not defined

In [ ]:
# TRAINING CHANNEL

for epoch in range(1):  # loop over the dataset multiple times
    for i, data in enumerate(trainloader, 0):
        # limit training time for debugging purposes
        if i > 5:
            break

        #since we are not interested in inference, loss and latency can be calculated in one training loop
        calculate_regularizer_metrics(model_components,inputs,labels)
        # calculate_FLOPS()
        print(i)

print('Finished Training')

In [ ]:
print(loss_across_epochs)

In [ ]:
print(latency_across_epochs)

In [ ]:
######### PLOTTING FIGURES ########

# -> utils/plotting_utils.py
columns=["No Regularizer","L1 Regularizer"]
df = pd.DataFrame(loss_across_epochs, columns=columns)
print(df)
plot = df.plot(title="Regularizer vs Loss")
plot.set(xlabel="Time/Iteration", ylabel="Loss")

In [ ]:
columns2=["No Regularizer","L1 Regularizer"]
df2 = pd.DataFrame(latency_across_epochs, columns=columns2)
print(df2)

plot = df2.plot(title="Regularizer vs Latency")
plot.set(xlabel="Time/Iteration", ylabel="Latency (s)")

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:
# -> utils/evaluation_utils.py

correct = 0
total = 0
net = Net()
net.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
# -> utils/evaluation_utils.py

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))